# Timetable Optimization - Visualization Demo

This notebook demonstrates the visualization capabilities of the timetable optimization system.

In [ ]:
import sys
sys.path.insert(0, '/home/emmanuelabayor/projects/timetable-sa/python/src')

import matplotlib.pyplot as plt
import numpy as np
from timetable_sa.examples.timetabling import (
    create_initial_state,
    generate_default_time_slots,
    SimulatedAnnealing,
    NoRoomConflict,
    NoLecturerConflict,
    NoProdiConflict,
    Compactness,
    ChangeTimeSlot,
    ChangeRoom,
    SwapClasses,
)
from timetable_sa.examples.timetabling.data.excel_loader import load_data_uisi
from timetable_sa.core.interfaces import SAConfig

plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## 1. Load Data and Create Initial State

In [ ]:
# Load sample data
rooms, lecturers, classes = load_data_uisi()
print(f"Loaded {len(rooms)} rooms, {len(lecturers)} lecturers, {len(classes)} classes")

# Generate time slots
time_slots = generate_default_time_slots()
print(f"Generated {len(time_slots)} time slots")

# Create initial state with a small subset for demo
sample_classes = classes[:20]  # Use first 20 classes for demo
state = create_initial_state(sample_classes, rooms, lecturers, time_slots)
print(f"Created initial state with {len(state.schedule)} scheduled classes")

## 2. Run Optimization with History Tracking

In [ ]:
# Define constraints
constraints = [
    NoRoomConflict(),
    NoLecturerConflict(),
    NoProdiConflict(),
    Compactness(),
]

# Define move generators
move_generators = [
    ChangeTimeSlot(),
    ChangeRoom(),
    SwapClasses(),
]

# Configure SA
config = SAConfig(
    initial_temperature=1000.0,
    min_temperature=0.1,
    cooling_rate=0.995,
    max_iterations=1000,
)

# Create SA instance
sa = SimulatedAnnealing(state, constraints, move_generators, config)

# Run optimization
print("Running optimization...")
result = sa.solve()
print(f"Optimization complete!")
print(f"Final fitness: {result.fitness:.4f}")
print(f"Hard violations: {result.hard_violations}")
print(f"Soft violations: {result.soft_violations}")
print(f"Iterations: {result.iterations}")

## 3. Fitness Over Iterations

In [ ]:
# Plot fitness over iterations
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Fitness history
ax1 = axes[0]
ax1.plot(result.fitness_history, color='blue', linewidth=1.5)
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Fitness Score')
ax1.set_title('Fitness Over Iterations')
ax1.grid(True, alpha=0.3)
ax1.axhline(y=1.0, color='green', linestyle='--', alpha=0.7, label='Optimal (1.0)')
ax1.legend()

# Temperature over iterations
ax2 = axes[1]
ax2.plot(result.temperature_history, color='orange', linewidth=1.5)
ax2.set_xlabel('Iteration')
ax2.set_ylabel('Temperature')
ax2.set_title('Temperature Decay (Cooling Schedule)')
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')

plt.tight_layout()
plt.savefig('/home/emmanuelabayor/projects/timetable-sa/python/visualization/fitness_history.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: visualization/fitness_history.png")

## 4. Violations Reduction

In [ ]:
# Plot violations over iterations
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Hard violations
ax1 = axes[0]
ax1.plot(result.hard_violations_history, color='red', linewidth=1.5)
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Hard Violations')
ax1.set_title('Hard Constraint Violations Reduction')
ax1.grid(True, alpha=0.3)

# Soft violations
ax2 = axes[1]
ax2.plot(result.soft_violations_history, color='blue', linewidth=1.5)
ax2.set_xlabel('Iteration')
ax2.set_ylabel('Soft Violations')
ax2.set_title('Soft Constraint Violations Reduction')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/home/emmanuelabayor/projects/timetable-sa/python/visualization/violations_history.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: visualization/violations_history.png")

## 5. Room Utilization Heatmap

In [ ]:
# Create room utilization heatmap
from timetable_sa.examples.timetabling.utils import group_by_room, group_by_day

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
periods = list(range(1, 13))

# Build utilization matrix
room_codes = [r.code for r in rooms[:10]]  # Top 10 rooms
utilization = np.zeros((len(room_codes), len(days) * len(periods)))

for entry in result.state.schedule:
    room_idx = room_codes.index(entry.room) if entry.room in room_codes else -1
    if room_idx >= 0:
        day_idx = days.index(entry.time_slot.day)
        period_idx = entry.time_slot.period - 1
        slot_idx = day_idx * len(periods) + period_idx
        if slot_idx < utilization.shape[1]:
            utilization[room_idx, slot_idx] = 1

# Plot heatmap
fig, ax = plt.subplots(figsize=(16, 8))

im = ax.imshow(utilization, cmap='YlOrRd', aspect='auto')

# Set labels
ax.set_yticks(range(len(room_codes)))
ax.set_yticklabels(room_codes)

# Set x-ticks for day/period
tick_positions = [i * len(periods) + 5 for i in range(len(days))]
ax.set_xticks(tick_positions)
ax.set_xticklabels(days)

# Add period grid lines
for i in range(1, len(days)):
    ax.axvline(x=i * len(periods) - 0.5, color='white', linewidth=1)

ax.set_xlabel('Day')
ax.set_ylabel('Room')
ax.set_title('Room Utilization Heatmap')

plt.colorbar(im, ax=ax, label='Occupied (1) / Empty (0)')
plt.tight_layout()
plt.savefig('/home/emmanuelabayor/projects/timetable-sa/python/visualization/room_utilization.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: visualization/room_utilization.png")

## 6. Operator Statistics

In [ ]:
# Plot operator statistics
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

operators = list(result.operator_stats.keys())
attempts = [result.operator_stats[op]['attempts'] for op in operators]
accepted = [result.operator_stats[op]['accepted'] for op in operators]

# Attempts vs Accepted
ax1 = axes[0]
x = np.arange(len(operators))
width = 0.35
bars1 = ax1.bar(x - width/2, attempts, width, label='Attempts', color='steelblue')
bars2 = ax1.bar(x + width/2, accepted, width, label='Accepted', color='forestgreen')
ax1.set_xlabel('Operator')
ax1.set_ylabel('Count')
ax1.set_title('Move Generator Usage')
ax1.set_xticks(x)
ax1.set_xticklabels(operators, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Success rate
ax2 = axes[1]
success_rates = []
for op in operators:
    stats = result.operator_stats[op]
    rate = (stats['accepted'] / stats['attempts'] * 100) if stats['attempts'] > 0 else 0
    success_rates.append(rate)

bars = ax2.bar(operators, success_rates, color='coral')
ax2.set_xlabel('Operator')
ax2.set_ylabel('Success Rate (%)')
ax2.set_title('Move Generator Success Rate')
ax2.set_xticklabels(operators, rotation=45, ha='right')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, rate in zip(bars, success_rates):
    height = bar.get_height()
    ax2.annotate(f'{rate:.1f}%',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom')

plt.tight_layout()
plt.savefig('/home/emmanuelabayor/projects/timetable-sa/python/visualization/operator_stats.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: visualization/operator_stats.png")

## 7. Schedule by Day

In [ ]:
# Create schedule visualization by day
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]

for idx, day in enumerate(days):
    ax = axes[idx]
    
    # Get classes for this day
    day_classes = [e for e in result.state.schedule if e.time_slot.day == day]
    
    if not day_classes:
        ax.text(0.5, 0.5, 'No classes', ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'{day}')
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        continue
    
    # Plot classes as horizontal bars
    y_positions = []
    colors = plt.cm.Set3(np.linspace(0, 1, len(day_classes)))
    
    for i, entry in enumerate(day_classes):
        start = entry.time_slot.period
        duration = min(entry.sks, 13 - start)
        ax.barh(i, duration, left=start, color=colors[i], edgecolor='black', height=0.8)
        
        # Add class label
        label = f"{entry.class_id}\n{entry.room}"
        ax.text(start + duration/2, i, label, ha='center', va='center', fontsize=8)
        
        y_positions.append(i)
    
    ax.set_yticks(y_positions)
    ax.set_yticklabels([e.kelas for e in day_classes], fontsize=8)
    ax.set_xlabel('Period')
    ax.set_title(f'{day} ({len(day_classes)} classes)')
    ax.set_xlim(0.5, 12.5)
    ax.set_xticks(range(1, 13))
    ax.grid(True, alpha=0.3, axis='x')

# Hide the 6th subplot if not needed
axes[5].axis('off')

plt.suptitle('Weekly Schedule Overview', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('/home/emmanuelabayor/projects/timetable-sa/python/visualization/weekly_schedule.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: visualization/weekly_schedule.png")

## Summary

In [ ]:
print("=" * 60)
print("VISUALIZATION SUMMARY")
print("=" * 60)
print(f"Initial Hard Violations: {result.initial_hard_violations}")
print(f"Final Hard Violations: {result.hard_violations}")
print(f"Improvement: {result.initial_hard_violations - result.hard_violations} violations eliminated")
print()
print(f"Total Iterations: {result.iterations}")
print(f"Final Fitness: {result.fitness:.4f}")
print(f"Execution Time: {result.duration_seconds:.2f} seconds")
print()
print("Generated Visualizations:")
print("  - fitness_history.png")
print("  - violations_history.png")
print("  - room_utilization.png")
print("  - operator_stats.png")
print("  - weekly_schedule.png")
print("=" * 60)